In [ ]:
import sqlite3


connection=sqlite3.connect('data.db')
connection.close()






CURSOR :
    All operations in sqlite are made by cursors and not by connect object itself.
    Because, we can have only one connection at a time but multiple cursors allow us to perform multiple database operations. 
    Example : One to read some data and other to write data..
        
COMMIT :
    It means save the result to the disc.
    

In [ ]:
import sqlite3


def create_book():
    connection=sqlite3.connect('data.db')
    cursor=connection.cursor()
    
    
    cursor.execute('CREATE TABLE IF NOT EXISTS books(name text primary key,author text,read integer)')  
                                                                              # only 5 data types: null,integer,real,text,blob
                                                                              # blob means image, pdf etc i.e stored exactly 
                                                                              # as it was input !
                                                                              # primary key is main identifier column
    
    
    connection.commit()
    connection.close()
        

    
    
def book_list():
    connection=sqlite3.connect('data.db')
    cursor=connection.cursor()
    
    cursor.execute('SELECT * FROM books')
     #books=cursor.fetchall()  this will give me a list of tuples. So its better to use list comprehension to turn it into 
                                                                    #dictionaries for convenience.
    
    books_new=[{'name':row[0],'author':row[1],'read':row[2]} for row in cursor.fetchall()] #list comprehension
    
    connection.close()
    return books_new
    
    
    


def add_book():
    name=input('Enter the name of the book : ')
    author=input('Enter the name of the author : ')
    
    connection=sqlite3.connect('data.db')
    cursor=connection.cursor()
    
    
    
    # cursor.execute(f'INSERT INTO books VALUES("{name}","{author}",0))  - not recommended.. if there is a direct replacement of
    # a string into a query , then there is apossiblity of SEQUEL INJECTION ATTACK ! ",0);DROP TABLE books; . SO to avoid it
    
    cursor.execute('INSERT INTO books VALUES (?,?,0)', (name,author))
    
    connection.commit()
    connection.close()
                   


        
    
def mark_read():
    entry=input('Enter the name of the book you want to mark as read : ')
    connection=sqlite3.connect('data.db')
    cursor=connection.cursor()
    cursor.execute('UPDATE books SET read=1 WHERE name=?',(entry,))
    connection.commit()
    connection.close()

   
    
    
    # UPDATE table_name SET column='...' where id=1 or any condition like where name='....' OR/AND <= >=
                   
    #because the replacement has to be a tuple....
    
    
    

        
def delete_book():
    name=input('Enter the name of the book you want to delete : ')
    connection=sqlite3.connect('data.db')
    cursor=connection.cursor()
    
    cursor.execute('DELETE FROM books WHERE name=?',(name,))
    
    
    
    
    connection.commit()
    connection.close()
    
def menu():
    create_book()
    sel = input("Enter 'a' to Add a book,"
                "'s' to see the list of added books,"
               "'d' to delete a book,"
               "'m' to mark a book as read,"
               "'q' to quit! : ")
    
    while sel!='q':
        if sel=='a':
            add_book()
        
        elif sel=='s':
            books=book_list()
            for book in books:
                book['read']='YES' if book['read'] else 'NO'
                print(f"{book['name']},{book['author']},{book['read']}")
            
        elif sel=='d':
            delete_book()
        
        
        elif sel=='m':
            mark_read()
            
        else:
            print('UNKNOWN COMMAND!')
            
        sel = input("Enter 'a' to dd a book,"
                          "'s' to see the list of added books,"
                          "'d' to delete a book,"
                          "'m' to mark a book as read,"
                          "'q' to quit! : ")
        
        
menu()

In [ ]:
#CONTEXT MANAGERS:
Example: with ..... as .... : 
        pass
    
example 2: In previous program, we used commands to open connection,connect,commit and close very frequently..... now this can b
    be made into custom context managers....
    




In [ ]:
#Connection.py


import sqlite3

class Dbconn: #to wirte a with context manager, we need 2 dunder methods
    
    def __init__(self,host):
        self.connection = None
        self.host=host
        
    def __enter__(self):
        self.connection= sqlite3.connect(self.host)
        return self.connection
    
    def __exit__(self,exc_type,exc_val,exc_tb):
        if exc_type or exc_val or exc_tb:
            self.connection.close()
        else:
            self.connection.commit()
            self.connection.close()
    
    
    
def mark_read():
    with Dbconn('data.db') as connection:
        cursor=connection.cursor()
        cursor.execute('UPDATE books SET read=1 WHERE name=?',(entry,))

        
# exit : Exception type, exception value and exception traceback are must arguments.. Say, if there happens any runtime error
# while executing the context manager in main program , the program will still execute the exit method and then raise an error.